# **Football Player Performance Dashboard**

# **1. Objectives**

## **1.1. Provide Interactive Monitoring**
Offer an interactive tool for football managers to monitor player performance in real-time.

## **1.2. Analyze Performance Metrics**
Visualize and analyze various performance metrics to assess player effectiveness.

## **1.3. Compare Across Entities**
Compare player performance across different countries, leagues, and clubs for broader insights.

## **1.4. Track Player Development**
Track and visualize player development over time to monitor growth and progress.

## **1.5. Identify Strengths and Weaknesses**
Highlight individual player strengths and weaknesses to aid in targeted development.

## **1.6. Predicting Future Valuation**
Predicting future valuation/aqcuisition cost of players using Regression Analysis.

# **2. Key Functionalities**

## **2.1. Filtering by League, Club, and Position**
Allows managers to focus on specific groups of players.

## **2.2. Player Performance Overview**
Visualizes key performance metrics for players.

## **2.3. Country-wise Analysis**
Compares the overall performance of players from different countries.

## **2.4. Individual Player Analysis**
Provides detailed metrics and suggestions for improvement for selected players.

In [1]:
# @title Pre-requisitie Libraries


! pip install dash dash-table plotly pandas
!pip install dash pandas plotly scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 22.8 MB/s eta 0:00:00


In [4]:
# @title Dashboard_Code


import dash
from dash import dcc, html, Input, Output
import dash_table
import pandas as pd
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
file_path = '/content/drive/MyDrive/male_players_football.csv'  # Update this path to your actual file path
df = pd.read_csv(file_path)

# Filter out any rows where 'league_name' or 'nationality_name' is null
df = df[df['league_name'].notna() & df['nationality_name'].notna()]

# Initialize Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Football Player Performance Dashboard"),

    html.Div([
        html.Label("Select Country"),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': i, 'value': i} for i in df['nationality_name'].unique()],
            value=df['nationality_name'].unique()[0]
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),

    html.Div([
        html.Label("Select League"),
        dcc.Dropdown(
            id='league-dropdown'
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),

    html.Div([
        html.Label("Select Club"),
        dcc.Dropdown(id='club-dropdown')
    ], style={'width': '48%', 'display': 'inline-block'}),

    html.Div([
        html.Label("Select Position"),
        dcc.Dropdown(id='position-dropdown')
    ], style={'width': '48%', 'display': 'inline-block', 'margin-top': '10px'}),

    dash_table.DataTable(id='player-stats'),

    dcc.Graph(id='player-performance'),

    html.Div([
        html.Label("Select Player"),
        dcc.Dropdown(id='player-dropdown')
    ], style={'width': '48%', 'display': 'inline-block', 'margin-top': '10px'}),

    html.Div(id='player-details'),

    html.H3("Suggestions for Improvement"),
    html.Div(id='improvement-suggestions'),

    html.H3("Predicted Future Value and Other Metrics"),
    html.Div(id='predicted-value')
])

@app.callback(
    Output('league-dropdown', 'options'),
    Output('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def set_league_options(selected_country):
    if selected_country:
        leagues = df[df['nationality_name'] == selected_country]['league_name'].unique()
        return [{'label': i, 'value': i} for i in leagues], leagues[0]
    return [], None

@app.callback(
    Output('club-dropdown', 'options'),
    Output('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def set_club_options(selected_league, selected_country):
    if selected_league and selected_country:
        clubs = df[(df['league_name'] == selected_league) &
                   (df['nationality_name'] == selected_country)]['club_name'].unique()
        return [{'label': i, 'value': i} for i in clubs], clubs[0]
    return [], None

@app.callback(
    Output('position-dropdown', 'options'),
    Output('position-dropdown', 'value'),
    Input('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def set_position_options(selected_club, selected_league, selected_country):
    if selected_club and selected_league and selected_country:
        positions = df[(df['club_name'] == selected_club) &
                       (df['league_name'] == selected_league) &
                       (df['nationality_name'] == selected_country)]['club_position'].unique()
        return [{'label': i, 'value': i} for i in positions], positions[0]
    return [], None

@app.callback(
    Output('player-stats', 'data'),
    Output('player-stats', 'columns'),
    Input('position-dropdown', 'value'),
    Input('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def update_player_table(selected_position, selected_club, selected_league, selected_country):
    if selected_position and selected_club and selected_league and selected_country:
        filtered_df = df[(df['club_position'] == selected_position) &
                         (df['club_name'] == selected_club) &
                         (df['league_name'] == selected_league) &
                         (df['nationality_name'] == selected_country)]
        columns = [{"name": i, "id": i} for i in ['short_name', 'age', 'overall', 'potential', 'value_eur', 'wage_eur', 'height_cm', 'weight_kg']]
        data = filtered_df.to_dict('records')
        return data, columns
    return [], []

@app.callback(
    Output('player-performance', 'figure'),
    Input('position-dropdown', 'value'),
    Input('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def update_performance_graph(selected_position, selected_club, selected_league, selected_country):
    if selected_position and selected_club and selected_league and selected_country:
        filtered_df = df[(df['club_position'] == selected_position) &
                         (df['club_name'] == selected_club) &
                         (df['league_name'] == selected_league) &
                         (df['nationality_name'] == selected_country)]
        fig = go.Figure()
        for metric in ['overall', 'potential', 'pace', 'shooting',
                       'passing', 'dribbling', 'defending', 'physic']:
            fig.add_trace(go.Bar(
                x=filtered_df['short_name'],
                y=filtered_df[metric],
                name=metric
            ))
        fig.update_layout(barmode='group', title="Player Performance Overview")
        return fig
    return {}

@app.callback(
    Output('player-dropdown', 'options'),
    Output('player-dropdown', 'value'),
    Input('position-dropdown', 'value'),
    Input('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def set_player_options(selected_position, selected_club, selected_league, selected_country):
    if selected_position and selected_club and selected_league and selected_country:
        players = df[(df['club_position'] == selected_position) &
                     (df['club_name'] == selected_club) &
                     (df['league_name'] == selected_league) &
                     (df['nationality_name'] == selected_country)]['short_name'].unique()
        return [{'label': i, 'value': i} for i in players], players[0]
    return [], None

@app.callback(
    Output('player-details', 'children'),
    Input('player-dropdown', 'value'),
    Input('position-dropdown', 'value'),
    Input('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def update_player_details(selected_player, selected_position, selected_club, selected_league, selected_country):
    if selected_player:
        player_data = df[(df['short_name'] == selected_player) &
                         (df['club_position'] == selected_position) &
                         (df['club_name'] == selected_club) &
                         (df['league_name'] == selected_league) &
                         (df['nationality_name'] == selected_country)].iloc[0]
        return html.Div([
            html.H4(f"Player: {player_data['short_name']}"),
            html.P(f"Age: {player_data['age']}"),
            html.P(f"Overall: {player_data['overall']}"),
            html.P(f"Potential: {player_data['potential']}"),
            html.P(f"Value: €{player_data['value_eur']:,.2f}"),
            html.P(f"Wage: €{player_data['wage_eur']:,.2f}"),
            html.P(f"Height: {player_data['height_cm']} cm"),
            html.P(f"Weight: {player_data['weight_kg']} kg")
        ])
    return "No player selected."

@app.callback(
    Output('improvement-suggestions', 'children'),
    Input('player-dropdown', 'value'),
    Input('position-dropdown', 'value'),
    Input('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def update_improvement_suggestions(selected_player, selected_position, selected_club, selected_league, selected_country):
    if selected_player:
        player_data = df[(df['short_name'] == selected_player) &
                         (df['club_position'] == selected_position) &
                         (df['club_name'] == selected_club) &
                         (df['league_name'] == selected_league) &
                         (df['nationality_name'] == selected_country)].iloc[0]
        improvement_areas = ['weak_foot', 'skill_moves', 'international_reputation',
                             'attacking_crossing', 'mentality_vision']
        suggestions = html.Ul([html.Li(f"{area}: {player_data[area]}") for area in improvement_areas])
        return suggestions
    return "No player selected."

@app.callback(
    Output('predicted-value', 'children'),
    Input('player-dropdown', 'value'),
    Input('position-dropdown', 'value'),
    Input('club-dropdown', 'value'),
    Input('league-dropdown', 'value'),
    Input('country-dropdown', 'value'))
def predict_future_value(selected_player, selected_position, selected_club, selected_league, selected_country):
    if selected_player:
        player_data = df[(df['short_name'] == selected_player) &
                         (df['club_position'] == selected_position) &
                         (df['club_name'] == selected_club) &
                         (df['league_name'] == selected_league) &
                         (df['nationality_name'] == selected_country)].iloc[0]

        # Features for the regression model: age, overall, potential, and selected areas
        features = np.array([
            player_data['age'],
            player_data['overall'],
            player_data['potential'],
            player_data['weak_foot'],
            player_data['skill_moves'],
            player_data['international_reputation'],
            player_data['attacking_crossing'],
            player_data['mentality_vision']
        ]).reshape(1, -1)

        # Dummy regression model (replace with a trained model)
        model = LinearRegression()

        # Simulate training data (replace with actual training data)
        X_train = np.random.randint(15, 90, size=(100, 8))  # Example feature data
        y_train_value = np.random.randint(1000000, 100000000, size=(100,))  # Example value data
        y_train_wage = np.random.randint(10000, 500000, size=(100,))  # Example wage data
        y_train_overall = np.random.randint(60, 99, size=(100,))  # Example overall rating data

        model.fit(X_train, y_train_value)
        predicted_value = model.predict(features)

        model.fit(X_train, y_train_wage)
        predicted_wage = model.predict(features)

        model.fit(X_train, y_train_overall)
        predicted_overall = model.predict(features)

        return html.Div([
            html.P(f"Predicted Future Value: €{predicted_value[0]:,.2f}"),
            html.P(f"Predicted Future Wage: €{predicted_wage[0]:,.2f}"),
            html.P(f"Predicted Future Overall Rating: {predicted_overall[0]:.2f}")
        ])
    return "No player selected."

if __name__ == '__main__':
    app.run_server(debug=True)


Mounted at /content/drive


<ipython-input-4-9f4a78d8d43e>:18: DtypeWarning:

Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.



<IPython.core.display.Javascript object>